# Análisis iniciativa RCI hasta 60

Este cuaderno tiene un análisis de la iniciativa RCI hasta 60. Se busca identificar qué hay que modificar para tener la TPP objetivo.

## Análisis descriptivo

In [1]:
# importar librerias
import pandas as pd
import seaborn as sn

In [2]:
cosechas = pd.read_excel(r"C:\Users\mariajose_chinchilla\Downloads\Cosechas Julio24.xlsx")

Primero, revisaremos a aquellos clientes que llevan su RCI arriba de 50, para evaluar qué iniciativas se están aplicando.

In [3]:
rci = cosechas[cosechas["rciInternoFinalDeudor"] > 50]
prop = len(rci) / len(cosechas)
prop_monto = rci["MONTO_DESEMBOLSADO"].sum() / cosechas["MONTO_DESEMBOLSADO"].sum()
prop_neto = rci["DESEMBOLSO_NETO"].sum() / cosechas["DESEMBOLSO_NETO"].sum()
print(f"La proporción de clientes con esta condición es de {prop}.")
print(f"La proporción en monto bruto es {prop_monto}")
print(f"La proporción en monto neto es {prop_neto}")

La proporción de clientes con esta condición es de 0.3709304394235901.
La proporción en monto bruto es 0.4252183330254854
La proporción en monto neto es 0.36934074553236634


Una buena pregunta a responder es cuál hubiese sido la TPP si todos estos créditos que llevaron el RCI arriba de 50 no hubieran ocurrido.

In [4]:
tpp_actual = sum(cosechas["MONTO_DESEMBOLSADO"] * cosechas["TASA_ORIGINAL"]) / sum(cosechas["MONTO_DESEMBOLSADO"])
# quitar los creditos de RCI arriba de 50
mascara_rci = cosechas["rciInternoFinalDeudor"] > 50
tpp_sin_iniciativa = sum(cosechas.loc[~mascara_rci]["MONTO_DESEMBOLSADO"] * cosechas.loc[~mascara_rci]["TASA_ORIGINAL"]) / sum(cosechas.loc[~mascara_rci]["MONTO_DESEMBOLSADO"])
tpp_iniciativa = sum(cosechas.loc[mascara_rci]["MONTO_DESEMBOLSADO"] * cosechas.loc[mascara_rci]["TASA_ORIGINAL"]) / sum(cosechas.loc[mascara_rci]["MONTO_DESEMBOLSADO"]) 
print(f"La TPP actual es de {tpp_actual} y sin los créditos con RCI alto hubiese sido {tpp_sin_iniciativa}")
print(f"La TPP de los créditos con RCI alto es de {tpp_iniciativa}")

La TPP actual es de 16.999463204331942 y sin los créditos con RCI alto hubiese sido 17.239446714271416
La TPP de los créditos con RCI alto es de 16.67506956657894


In [5]:
# flagear las iniciativas
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["Cumple el 15%"] == "Cumple") & (cosechas["ES_RECREDITO"] == "NO"), "PRECIO APLICADO"] = "BIENESTAR"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["Cumple el 15%"] != "Cumple") & (cosechas["BANRURAL CONSOLIDA"] == "SI"), "PRECIO APLICADO"] = "SALUD BANRURAL"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["Cumple el 15%"] != "Cumple") & (cosechas["BANRURAL CONSOLIDA"] != "SI") & (cosechas["PRODUCTO"] == "SALUD"), "PRECIO APLICADO"] = "SALUD OTROS BANCOS"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["Cumple el 15%"] != "Cumple") & (cosechas["BANRURAL CONSOLIDA"] != "SI") & (cosechas["PRODUCTO"] == "LIQUIDEZ"), "PRECIO APLICADO"] = "LIQUIDEZ BANRURAL"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["conSaldoCreditoExt"] > 0) & (cosechas["Cumple el 15%"] == "Cumple"), "PRECIO APLICADO"] = "BIENESTAR"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["conSaldoCreditoExt"] > 0) & (cosechas["Cumple el 15%"] != "Cumple"), "PRECIO APLICADO"] = "OTROS BANCOS CREDITO NUEVO"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["conSaldoCreditoExt"] == 0) & (cosechas["Cumple el 15%"] == "Cumple"), "PRECIO APLICADO"] = "BIENESTAR"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["conSaldoCreditoExt"] == 0) & (cosechas["Cumple el 15%"] != "Cumple"), "PRECIO APLICADO"] = "CREDITO PLUS"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["conSaldoCreditoExt"] == 0) & (cosechas["Cumple el 15%"] == "Cumple"), "PRECIO APLICADO"] = "BIENESTAR"
cosechas.loc[(cosechas["TIPO_CLIENTE"] == "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI") & (cosechas["Cumple el 15%"] == "Cumple") & (cosechas["ES_RECREDITO"] == "SI"), "PRECIO APLICADO"] = "CVF"

# ahora para cliente nuevo y reactivado
cosechas.loc[(cosechas["TIPO_CLIENTE"] != "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["BANRURAL CONSOLIDA"] == "SI"), "PRECIO APLICADO"] = "SALUD BANRURAL"
cosechas.loc[(cosechas["TIPO_CLIENTE"] != "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["BANRURAL CONSOLIDA"] != "SI") & (cosechas["PRODUCTO"] == "SALUD"), "PRECIO APLICADO"] = "SALUD OTROS BANCOS"
cosechas.loc[(cosechas["TIPO_CLIENTE"] != "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] == "SI") & (cosechas["BANRURAL CONSOLIDA"] == "SI") & (cosechas["PRODUCTO"] == "LIQUIDEZ"), "PRECIO APLICADO"] = "LIQUIDEZ BANRURAL"
cosechas.loc[(cosechas["TIPO_CLIENTE"] != "EXISTENTE") & (cosechas["BANRURAL ES CLIENTE"] != "SI"), "PRECIO APLICADO"] = "BIENESTAR"

In [6]:
# Ver qué iniciativas están involucradas en la tasa independiente del RCI alto
cosechas.loc[(mascara_rci) & (cosechas["TipoPago"] == "REMESA")]["PRECIO APLICADO"].unique()

array(['BIENESTAR', 'nan', 'CVF'], dtype=object)

Esto nos lleva a concluir que los créditos con RCI alto sí impactan negativamente en la TPP, aunque su impacto no es altamente significativo. Un punto de mejora para el aumento de la TPP es el monto bruto que cada clúster representa, donde por clúster entendemos iniciativa comercial.
    

In [7]:
props = []
monto_total = cosechas.loc[(mascara_rci) & (cosechas["TipoPago"] == "REMESA")]["MONTO_DESEMBOLSADO"].sum()
for iniciativa in cosechas.loc[mascara_rci]["PRECIO APLICADO"].unique().tolist():
    prop = sum(cosechas.loc[(mascara_rci) & (cosechas["PRECIO APLICADO"] == iniciativa) & (cosechas["TipoPago"] == "REMESA")]["MONTO_DESEMBOLSADO"]) / monto_total
    props.append((prop, iniciativa))

props = sorted(props, reverse=True)

for i in range(len(props)):
    print(f"{props[i][1]} representa la {props[i][0]} proporción del monto bruto en créditos con RCI altos.")

nan representa la 0.7342559186859318 proporción del monto bruto en créditos con RCI altos.
CVF representa la 0.13791157515929 proporción del monto bruto en créditos con RCI altos.
BIENESTAR representa la 0.12783250615477826 proporción del monto bruto en créditos con RCI altos.


## Analizar clientes que se van por CVF